# Heuristic Analysis for Minimax AI in Othello

In this notebook, we explore the most effective **heuristics** for guiding the Minimax algorithm. 

Several AI agents, each employing different combinations of heuristics, engage in head-to-head matches. Through these matches, we identify the strongest heuristics that lead to superior AI performance and strategic gameplay.

---

## Theoretical Analysis

The theoretical analysis in this notebook focuses on match evaluation. Specifically, we explore which AI agents (in terms of their heursitics, heuristic weights, and depth settings) achieve the most wins. Based on these results, we infer the strategies that lead to superior Othello AI agents.   

Hypotheses:

- Corner control is the most important heuristic [1], since gaining corner squares are fundamental to Othello strategy [2]. 
- Combining all three heuristics (disc difference, mobility, and corner control) should yield the strongest AI [1].
- Agents that search **deeper** (i.e higher depth parameter) are stronger, since they "look further ahead".

Methodology:

- Several AI agents are assessed, each using different combinations of heuristics, via. **head-to-head matches**.
- To ensure fairness, every agent competes against every other agent *twice*, once as Black and once as White.
- To ensure fairness, every Minimax agent has depth set to three, i.e. $d=3$. Additionally, this is done for practicality, i.e. computational efficiency.
- Results naturally generate a **league structure**, akin to the common format used in football leagues worldwide, following a "double round-robin" basis.
- To maintain simplicity, all agents employ uniform weights. We reserve weight optimization for future analyses, i.e. fine-tuning AI performance.


### References

[1] Sannidhanam, A., & Muthukaruppan, A. (2004). '[An Analysis of Heuristics in Othello](https://courses.cs.washington.edu/courses/cse573/04au/Project/mini1/RUSSIA/Final_Paper.pdf)'.

[2] Rose, B. (2004). '[Othello: A Minute to Learn... A Lifetime to Master.](https://www.ffothello.org/livres/othello-book-Brian-Rose.pdf)' Anjar Co.

---

## Load Source Code

In [1]:
import os
import sys

# Calculate path to the src directory and append to sys.path
current_dir = os.path.dirname(os.path.abspath("Heuristic_Analysis.ipynb"))
project_root = os.path.dirname(os.path.dirname(current_dir))
sys.path.append(os.path.join(project_root, 'src'))

from game import Game
from board import SquareType
from player import Player, PlayerType
from state_evaluation import StateEvaluator, HeuristicType

## Initialise Players

In [2]:
# Agents dictionary
agents = {}
depth = 1

# Agent 1: Disc difference
custom_weights = {HeuristicType.DISC_DIFF: 1.0}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_1'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 2: Mobility
custom_weights = {HeuristicType.MOBILITY: 1.0}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_2'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 3: Corners
custom_weights = {HeuristicType.CORNERS: 1.0}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_3'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 4: Disc difference & Corners
custom_weights = {HeuristicType.DISC_DIFF: 0.5, HeuristicType.CORNERS: 0.5}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_4'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 5: Disc difference & Mobility
custom_weights = {HeuristicType.DISC_DIFF: 0.5, HeuristicType.MOBILITY: 0.5}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_5'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 6: Mobility & Corners
custom_weights = {HeuristicType.MOBILITY: 0.5, HeuristicType.CORNERS: 0.5}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_6'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

# Agent 7: All Heuristics
custom_weights = {
    HeuristicType.DISC_DIFF: 1/3,
    HeuristicType.MOBILITY: 1/3,
    HeuristicType.CORNERS: 1/3
}
state_eval = StateEvaluator(weights=custom_weights)
agents['agent_7'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, state_eval, depth),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, state_eval, depth)
}

## Match Function

- A function to simulate an Othello match between two agents.
- Returns all relevant information, i.e winnner, score etc.

In [3]:
def simulate_match(agent_black_name, agent_white_name):
    # Get agent using dictionary
    agent_black = agents.get(agent_black_name)
    agent_white = agents.get(agent_white_name)
    
    # Instantiate game instance
    player_black = agent_black["black"]
    player_white = agent_white["white"]
    game = Game(player_black, player_white)
    
    while not game.is_finished:
        game.get_player_move() 
        game.make_move()
        game.change_turn()
        game.update_valid_moves()
        game.update_scores()
        game.check_finished()
    
    game.determine_winner()

    return {
        'game_result': game.game_result,
        'black_score': game.black_score,
        'white_score': game.white_score,
        'agent_black': agent_black_name,
        'agent_white': agent_white_name
    }
    

from tqdm import tqdm 

def generate_league():
    league_results = []
    agent_names = list(agents.keys())

    for i in tqdm(range(len(agent_names))):
        for j in tqdm(range(i + 1, len(agent_names))):

            print(f"Playing: {agent_names[i]} vs. {agent_names[j]}")
            
            # Play each pair of agents twice, once as black and once as white
            result_1 = simulate_match(agent_names[i], agent_names[j])
            result_2 = simulate_match(agent_names[j], agent_names[i])

            # Store results
            league_results.append(result_1)
            league_results.append(result_2)

    return league_results

# Generate the league
league_results = generate_league()
league_results

  0%|                                                 | 0/6 [00:00<?, ?it/s]

Playing: agent_1 vs. agent_2



 17%|██████▊                                  | 1/6 [00:03<00:16,  3.38s/it]

Playing: agent_1 vs. agent_3



 33%|█████████████▋                           | 2/6 [00:05<00:09,  2.46s/it]

...

Playing: agent_5 vs. agent_7



  0%|                                                 | 0/1 [00:00<?, ?it/s]

Playing: agent_6 vs. agent_7



 86%|███████████████████████████████████▏     | 6/7 [01:27<00:11, 11.35s/it]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 7/7 [01:27<00:00, 12.54s/it]


[{'game_result': 'Black Wins',
  'black_score': 40,
  'white_score': 0,
  'agent_black': 'agent_1',
  'agent_white': 'agent_2'},
 {'game_result': 'Draw',
  'black_score': 32,
  'white_score': 32,
  'agent_black': 'agent_2',
  'agent_white': 'agent_1'},
 {'game_result': 'White Wins',
  'black_score': 19,
  'white_score': 45,
  'agent_black': 'agent_1',
  'agent_white': 'agent_3'},
 {'game_result': 'Black Wins',
  'black_score': 33,
  'white_score': 31,
  'agent_black': 'agent_3',
  'agent_white': 'agent_1'},
 {'game_result': 'White Wins',
  'black_score': 19,
  'white_score': 45,
  'agent_black': 'agent_1',
  'agent_white': 'agent_4'},
 {'game_result': 'White Wins',
  'black_score': 20,
  'white_score': 44,
  'agent_black': 'agent_4',
  'agent_white': 'agent_1'},
 {'game_result': 'White Wins',
  'black_score': 20,
  'white_score': 44,
  'agent_black': 'agent_1',
  'agent_white': 'agent_5'},
 {'game_result': 'Black Wins',
  'black_score': 40,
  'white_score': 24,
  'agent_black': 'agent_